In [ ]:
#Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
#I'm creating a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

# I found it to work best with 3 convolutional layers.

In [3]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [4]:
def train_happy_sad_model():
    DESIRED_ACCURACY = 0.999
    class myCallback(tf.keras.callbacks.Callback):
         def on_epoch_end(self, epochs, logs={}) :
                if(logs.get('acc') >= DESIRED_ACCURACY) :
                    print('\nReached 99.9% accuracy so cancelling training!')
                    self.model.stop_training = True
    callbacks = myCallback()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(100,100,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    from tensorflow.keras.optimizers import RMSprop
    model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['acc'])
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale = 1/255)
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/', 
        target_size = (100,100),
        batch_size = 80,
        class_mode='binary')
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=1,  
        epochs=20,
        verbose=1, 
        callbacks=[callbacks])
    return history.history['acc'][-1]

In [ ]:
train_happy_sad_model()

In [ ]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();

In [ ]:
%%javascript
IPython.notebook.session.delete();
window.onbeforeunload = null
setTimeout(function() { window.close(); }, 1000);